In [1]:
import pandas as pd
import logging
import os
from datetime import datetime, timedelta, time

# detailed debug & log info
logging.basicConfig(level=logging.INFO)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
logging.basicConfig(level=logging.DEBUG)
# workspace_directory = r'/home/ubuntu/Tryggvi/my-awesome-masters-project' #VM
# workspace_directory = r"C:/Users/tryggvisi/Documents/my-awesome-masters-project" #vinna
workspace_directory = r"C:/Users/trygg/Documents/Master_project" #home
os.chdir(workspace_directory)
#directory setup

### Convert to middletime to correct time zone

In [28]:
#ERA5
#Vindefjallen data
csv_file_path_2023_era5 = r"Data/Vindefjallen_data/Vindefjallen_cleaning/Vindefjallen_Granule_processing/vindefjallen_corner_coords_processing_2023_middletime_era5.csv"

df_measurements_era5 = pd.read_csv(csv_file_path_2023_era5) # Read the CSV file into a DataFrame

df_measurements_era5 = df_measurements_era5.sort_values(by='date')
df_measurements_era5 = df_measurements_era5.reset_index(drop=True)

#converts the middletime column to time object and then corrects for time zone and converts back to string
#calculate time difference between granule time and measurement time
def time_zone_converter(meas_date, middle_time):
    # Convert meas_date to datetime object
    meas_date = datetime.strptime(meas_date, '%Y-%m-%d') 
    
    # Convert middle_time to time object
    middle_time = time.fromisoformat(middle_time)
    # Create a dummy datetime object with today's date and middle_time
    dummy_date = datetime.combine(datetime.today(), middle_time)
    
    #granules from timezone UTC +00, while Sweden in UTC +01 at winter time, and UTC +02 at summer time
    if meas_date < datetime(meas_date.year, 3, 31):
        # Add 1 hour to dummy_date
        result_datetime = dummy_date - timedelta(hours=1)
    else:
        # Add 2 hours to dummy_date
        result_datetime = dummy_date - timedelta(hours=2)

    #convert dummy date to hours and minutes and to string
    granule_time = result_datetime.strftime('%H:%M:%S')
    return granule_time




### ERA5 new processing

In [54]:
#fyrir nýtt processing
granule_list = r'Data/Vindefjallen_data/Vindefjallen_cleaning/Vindefjallen_Granule_processing/list_of_slc_granules_2023_era5.csv'
granules_df = pd.read_csv(granule_list)
granules_df['granule_date'] = pd.to_datetime(granules_df['granule_date'])
granules_df['granule_date'] = granules_df['granule_date'].dt.strftime('%Y-%m-%d') #smá fiff til að laga date format

# # fyrir 100m
# clip_output_directory = workspace_directory + "/"+ r"Data/Processed_granules/processed_granules_clip_era5" 
# clip_output_directory = workspace_directory + "/"+ r"Data/Processed_granules/processed_granules_clip_era5_500" 
clip_output_directory = workspace_directory + "/"+ r"Data/Processed_granules/processed_granules_clip_era5_1000" 

clip_ending = '_clip_1000'


granule_paths_dict_era5 = {}
for i in range(len(granules_df)):
    granule_id = granules_df.loc[i, 'granule_ID']
    short_path = str(i)+'_'+ granules_df.loc[i, 'granule_date'] + clip_ending
    granule_paths_dict_era5[i] = { 
        "path_to_clip_granule": clip_output_directory + '/' + short_path, 
        'short_path_to_clip_granule': short_path,
        'granule_date': granules_df.loc[i, 'granule_date'],
        'granule_id': granule_id,
        'granule_time': granules_df.loc[i, 'startTime'][11:19],
        'orbitProp': granules_df['orbitProp'][i],
        'pathNr': granules_df['pathNr'][i],
        'frameNr': granules_df['frameNr'][i]
    }
# print(granule_paths_dict[0]['granule_name'])
print(granule_paths_dict_era5[0]['path_to_clip_granule'])
print(len(granule_paths_dict_era5))

C:/Users/trygg/Documents/Master_project/Data/Processed_granules/processed_granules_clip_era5_1000/0_2023-03-28_clip_1000
31


In [55]:
#ERA5
#Vindefjallen data
csv_file_path_2023_era5 = r"Data/Vindefjallen_data/Vindefjallen_cleaning/Vindefjallen_Granule_processing/vindefjallen_corner_coords_processing_2023_middletime_era5.csv"

df_measurements_era5 = pd.read_csv(csv_file_path_2023_era5) # Read the CSV file into a DataFrame

df_measurements_era5 = df_measurements_era5.sort_values(by='date')
df_measurements_era5 = df_measurements_era5.reset_index(drop=True)

print(df_measurements_era5.loc[1, 'MiddleTime'], df_measurements_era5.loc[1, 'ID'])
for i in range(len(df_measurements_era5)):
    x = time_zone_converter(df_measurements_era5.loc[i, 'date'], df_measurements_era5.loc[i, 'MiddleTime'])
    df_measurements_era5.loc[i, 'MiddleTime'] = x
print(df_measurements_era5.loc[1, 'MiddleTime'])  

14:57:30 293
13:57:30


### Old processing

In [37]:
# fyrir old processing
granule_list = r'Data/Vindefjallen_data/Vindefjallen_cleaning/Vindefjallen_Granule_processing/list_of_slc_granules_2023.csv'
granules_df = pd.read_csv(granule_list)

# # fyrir 100m
# clip_output_directory = workspace_directory + "/"+ r"Data/Processed_granules/clipped_granules" 
# clip_output_directory = workspace_directory + "/"+ r"Data/Processed_granules/processed_granules_clip" 
# clip_ending = '_clip'
#fyrir 500m
# clip_output_directory = workspace_directory + "/"+ r"Data/Processed_granules/processed_granules_clip_500" 
# clip_ending = '_clip_500'
# #fyrir 1000m
clip_output_directory = workspace_directory + "/"+ r"Data/Processed_granules/processed_granules_clip_1000"
clip_ending = '_clip_1000'

granule_paths_dict_old = {}
for i in range(len(granules_df)):
    granule_id = granules_df.loc[i, 'granule_ID']
    short_path = str(i)+'_'+ granules_df.loc[i, 'date'] + clip_ending
    granule_paths_dict_old[i] = { 
        "path_to_clip_granule": clip_output_directory + '/' + short_path, 
        'short_path_to_clip_granule': short_path,
        'granule_date': granules_df.loc[i, 'date'],
        'granule_id': granule_id,
        'granule_time': granules_df.loc[i, 'startTime'][11:19],
        'orbitProp': granules_df['orbitProp'][i],
        'pathNr': granules_df['pathNr'][i],
        'frameNr': granules_df['frameNr'][i]
    }

print(len(granule_paths_dict_old))
granule_paths_dict_old[1]

57


{'path_to_clip_granule': 'C:/Users/trygg/Documents/Master_project/Data/Processed_granules/processed_granules_clip_1000/1_2023-03-25_clip_1000',
 'short_path_to_clip_granule': '1_2023-03-25_clip_1000',
 'granule_date': '2023-03-25',
 'granule_id': 'S1A_IW_SLC__1SDV_20230325T161511_20230325T161538_047803_05BE49_44F8',
 'granule_time': '16:15:11',
 'orbitProp': 'ASCENDING',
 'pathNr': 131,
 'frameNr': 214}

In [39]:
#old
#Vindefjallen data
csv_file_path_2023_old = r"Data/Vindefjallen_data/Vindefjallen_cleaning/Vindefjallen_Granule_processing/vindefjallen_corner_coords_processing_2023_middletime.csv"

df_measurements_old = pd.read_csv(csv_file_path_2023_old) # Read the CSV file into a DataFrame

df_measurements_old = df_measurements_old.sort_values(by='date')
df_measurements_old = df_measurements_old.reset_index(drop=True)
df_measurements_old.loc[493, 'MiddleTime'] = '09:30:00'
# print(df_measurements_old.to_markdown())
print(df_measurements_old.loc[3, 'MiddleTime'], df_measurements_old.loc[3, 'ID'])

for i in range(len(df_measurements_old)):
    x = time_zone_converter(df_measurements_old.loc[i, 'date'], df_measurements_old.loc[i, 'MiddleTime'])
    df_measurements_old.loc[i, 'MiddleTime'] = x
print(df_measurements_old.loc[1, 'MiddleTime'], df_measurements_old.loc[1, 'ID'])


13:52:30 3
09:50:00 286


## Main processing

In [44]:
def calculate_time_difference(MiddleTime, granule_date, original=False):
    MiddleTime = datetime.strptime(MiddleTime, "%H:%M:%S")
    granule_date = datetime.strptime(granule_date, "%H:%M:%S")
    if original: #check if its for Era5 or old
        if original == "plus":
            granule_date = granule_date + timedelta(hours=24)
        elif original == "minus":
            granule_date = granule_date - timedelta(hours=24)
    else:
        pass

    # Calculate the time difference in absoluate to avoid negative values
    time_diff = abs(MiddleTime - granule_date)
    
    total_seconds = time_diff.total_seconds()

    # Calculate hours, minutes
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)

    # Format the result
    time_diff = f"{hours:02d}:{minutes:02d}:00"
    return time_diff

Þessi kóða dálkur inniheldur megin virknina til að undirbúa skrár til point extraction.

1. keyrt í gengum hverja staka vindelfjallen snow depth mælingu ~500
2. keyrt í gegnum dict sem var útbúin áður sem inniheldur slóðann á allar möppurnar sem hýsa klippta raster granules (4 per mappa)
3. condition: tjekkað hvort að dagsetning mælingar passi við dagsetningu á granue (sem er í möppuheiti)
4. ef condition eru True; þá er náð í lista af öllum skrám í möppunni (4 alls)
5. keyrt í gegnum listann með skráarheitum; búinn til slóði á hverja skrá
6. í lokin er útbúinn listi sem inniheldur alla dálka frá vindelfjallen mælingum ásamt 4 slóðum á .tif rastera.


In [45]:
# Main solution. Works for both ERA5 and old
def granule_path_creater(df_measurements, granule_paths_dict, date_column_name):
    # Iterate over each measurement in df_measurements_era5
    measurement_plus_granulePath_list = []
    for index, measurement in df_measurements.iterrows():
        # Iterate over each granule in granule_paths_dict
        for i in granule_paths_dict:
            # Check if the date of the measurement matches the date in granule_paths_dict
            if measurement[date_column_name] == granule_paths_dict[i]["granule_date"]:
                file_list = os.listdir(granule_paths_dict[i]["path_to_clip_granule"])
                listi = []
                annar_listi = []
                # Iterate over each file in the granule's clip directory
                for file in file_list:
                    listi.append(
                        f"{granule_paths_dict[i]['path_to_clip_granule']}/{file}"
                    )
                if date_column_name == 'era5_new_date':
                    time_difference =  calculate_time_difference(
                            measurement["MiddleTime"],
                            granule_paths_dict[i]["granule_time"],
                            measurement["era5_plusminus"]
                        )
                else:
                    time_difference =  calculate_time_difference(
                            measurement["MiddleTime"],
                            granule_paths_dict[i]["granule_time"],
                        )
                annar_listi = [
                    granule_paths_dict[i]["granule_time"],
                    time_difference,
                    granule_paths_dict[i]["granule_id"],
                    granule_paths_dict[i]["orbitProp"],
                    granule_paths_dict[i]["pathNr"],
                    granule_paths_dict[i]["frameNr"],
                ]

                measurement_plus_granulePath_list.append(
                    measurement.tolist() + annar_listi + listi
                )
    new_column_names = list(df_measurements.columns) + [
        "granule_time",
        "time_difference",
        "granule_id",
        "orbitProp",
        "pathNr",
        "frameNr",
        "mask.tif",
        "angle.tif",
        "VH.tif",
        "VV.tif",
    ]
    global df
    df = pd.DataFrame(measurement_plus_granulePath_list)
    df.columns = new_column_names
    if date_column_name == 'era5_new_date':
        df = df.drop(columns='Unnamed: 0')
 
    return df

In [58]:
#testing
df = granule_path_creater(df_measurements_era5, granule_paths_dict_era5, 'era5_new_date')# granule_path_creater(df_measurements_old, granule_paths_dict_old, 'date')
# df = granule_path_creater(df_measurements_old, granule_paths_dict_old, 'date')# granule_path_creater(df_measurements_old, granule_paths_dict_old, 'date')


In [59]:
print(df.to_markdown())

|     |   ID |   OG_ID | Triangle   | Corner   | TriangleCorner   | date       | SnowDepth   |   SnowDepth_Noplus |   temperature |   x_4326 |   y_4326 |   x_3006 |      y_3006 | MiddleTime   | era5_new_date   | era5_plusminus   |   era5_precip |   era5_temp | granule_time   | time_difference   | granule_id                                                          | orbitProp   |   pathNr |   frameNr | mask.tif                                                                                                                                                                    | angle.tif                                                                                                                                                                     | VH.tif                                                                                                                                                                  | VV.tif                                                                     

In [60]:
csv_name = "values_to_point_clipGranule_fullpaths_era5_1000.csv"
final_path = r'Data/Vindefjallen_data/Vindefjallen_cleaning/exploratory_data_analysis/1_values_from_raster_python/' + csv_name
if os.path.exists(final_path):
    print(f'Error: File "{final_path}" already exists. Please choose a different file name.')
else:
    df.to_csv(r'Data/Vindefjallen_data/Vindefjallen_cleaning/exploratory_data_analysis/1_values_from_raster_python/' + csv_name, index=True)
    print('Success: File has been saved.')

Success: File has been saved.
